# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r'C:\Homework\python-API-challenge\WeatherPy\output_data\cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gaoping,30.7758,106.1029,83.50,46,100,3.91,CN,1686448042
1,1,stanley,54.8680,-1.6985,52.70,94,0,2.30,GB,1686447763
2,2,thompson,55.7435,-97.8558,73.56,25,20,10.36,CA,1686448025
3,3,port-aux-francais,-49.3500,70.2167,39.33,67,99,30.56,TF,1686448028
4,4,dhana,23.7500,78.8500,91.89,30,52,17.81,IN,1686448044


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
map_humidity = city_data_df.hvplot.points( 
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 2,
    frame_width = 800,
    frame_height = 600
    )

# # Display the map
map_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values
my_ideal_weather = city_data_df.loc[(city_data_df["Max Temp"]<=75)
                                    & (city_data_df["Max Temp"]>=65)
                                    & (city_data_df["Humidity"]>=55)
                                    & (city_data_df["Humidity"]<=60),:]

# Drop any rows with null values
my_ideal_weather.dropna()


# Display sample data
my_ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,t'aebaek,37.1759,128.9889,73.18,58,4,2.06,KR,1686448045
37,37,heihe,50.2441,127.4902,73.98,56,4,3.24,CN,1686448058
50,50,adamstown,-25.0660,-130.1015,69.10,55,76,14.90,PN,1686448064
285,285,portland,45.5234,-122.6762,74.52,58,20,10.36,US,1686447462
298,298,pulandian,39.3953,121.9669,71.31,58,55,8.70,CN,1686448175
421,421,fedorovka,53.6314,62.7078,65.41,57,18,8.86,KZ,1686448224
465,465,burao,9.5221,45.5336,74.05,58,100,19.66,SO,1686448243
466,466,fort st. john,56.2499,-120.8529,67.89,60,75,9.22,CA,1686448094
475,475,guercif,34.2257,-3.3536,72.01,60,15,1.72,MA,1686448246


### Step 3: Create a new DataFrame called `hotel_df`.

In [72]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.reset_index()

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,6,t'aebaek,KR,37.1759,128.9889,58,
1,37,heihe,CN,50.2441,127.4902,56,
2,50,adamstown,PN,-25.0660,-130.1015,55,
3,285,portland,US,45.5234,-122.6762,58,
4,298,pulandian,CN,39.3953,121.9669,58,
5,421,fedorovka,KZ,53.6314,62.7078,57,
6,465,burao,SO,9.5221,45.5336,58,
7,466,fort st. john,CA,56.2499,-120.8529,60,
8,475,guercif,MA,34.2257,-3.3536,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [88]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit": 1
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["categories"] = "accommodation.hotel"
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = (name_address).json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
t'aebaek - nearest hotel: 이스턴호텔
heihe - nearest hotel: Санкт-Петербург
adamstown - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
pulandian - nearest hotel: 清泉宫温泉会馆
fedorovka - nearest hotel: No hotel found
burao - nearest hotel: Liiban hotel burco ( ENG:Shibiin )
fort st. john - nearest hotel: Fort St. John Motor Inn
guercif - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
6,t'aebaek,KR,37.1759,128.9889,58,이스턴호텔
37,heihe,CN,50.2441,127.4902,56,Санкт-Петербург
50,adamstown,PN,-25.0660,-130.1015,55,No hotel found
285,portland,US,45.5234,-122.6762,58,the Hoxton
298,pulandian,CN,39.3953,121.9669,58,清泉宫温泉会馆
421,fedorovka,KZ,53.6314,62.7078,57,No hotel found
465,burao,SO,9.5221,45.5336,58,Liiban hotel burco ( ENG:Shibiin )
466,fort st. john,CA,56.2499,-120.8529,60,Fort St. John Motor Inn
475,guercif,MA,34.2257,-3.3536,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [91]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 2,
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)